# 20151230-predict-household-income-from-census

Related post:  
https://stharrold.github.io/20151230-predict-household-income-from-census.html

Purpose: Predict total annual household income.

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [212]:
# Import standard packages.
import collections
import json
import os
import pdb # TEST: Comment out pdb after testing.
import pprint
import sys
# Import installed packages.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Import local packages.
sys.path.insert(
    0,
    os.path.join(os.path.curdir, 'stharrold.github.io/content/static/dsdemos'))
# TEST: Comment out autoreload after testing.
%reload_ext autoreload
%autoreload 2
import dsdemos as dsd
# IPython magic.
%matplotlib inline

## Globals

In [3]:
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20151230-predict-household-income-from-census'
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')
path_acs = os.path.join(path_disk, r'www2-census-gov/programs-surveys/acs/')
path_csv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13hdc.csv') # 'hdc' = 'housing DC'
path_ddict = os.path.join(path_acs, r'tech_docs/pums/data_dict/PUMS_Data_Dictionary_2009-2013.txt')

## Extract-transform-load

**TODO:**
* Just use pandas. Acknowledge dask.

In [4]:
%%time
with open(path_csv) as fobj:
    nlines = sum(1 for _ in fobj)
print("{path}:".format(path=path_csv))
print("size (MB) = {size:.1f}".format(size=os.path.getsize(path_csv)/1e6))
print("num lines = {nlines}".format(nlines=nlines))
df = pd.read_csv(path_csv)
print("df RAM usage (MB) = {mem:.1f}".format(mem=df.memory_usage().sum()/1e6))

/mnt/disk-20151227t211000z/www2-census-gov/programs-surveys/acs/data/pums/2013/5-Year/ss13hdc.csv:
size (MB) = 13.5
num lines = 17501
df RAM usage (MB) = 28.7
CPU times: user 444 ms, sys: 20 ms, total: 464 ms
Wall time: 466 ms


In [5]:
percentiles = [0.1587, 0.5000, 0.8413] # +1 std. dev., mean/median, -1 std. dev. for normal dist.
df.describe(percentiles=percentiles, include='all')

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
count,6561.000000,17500,1.750000e+04,17500,17500.000000,17500.000000,17500,17500,17500.000000,17500.000000,...,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000
unique,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,17500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,999.282731,NaN,2.011068e+12,5,56.427371,37.764171,3,11,1039364.231657,1048478.770229,...,17.050857,17.043486,17.050000,17.049029,17.048000,17.051543,17.053200,17.047029,17.046971,17.051486
std,1085.174484,NaN,1.401911e+09,0,55.291036,55.358495,0,0,31877.254257,29598.269890,...,17.593886,17.740566,17.534604,17.555515,17.558942,17.574232,17.623017,17.802284,17.267472,17.710924
min,0.000000,NaN,2.009000e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15.9%,200.000000,NaN,2.009001e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,790.000000,NaN,2.011001e+12,5,101.000000,-9.000000,3,11,1035725.000000,1054614.000000,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
84.1%,1500.000000,NaN,2.013000e+12,5,104.000000,104.000000,3,11,1086032.000000,1085467.000000,...,30.000000,30.000000,30.000000,31.000000,30.000000,30.000000,30.000000,30.000000,31.000000,30.000000


In [6]:
ddict = dsd.census.parse_pumsdatadict(path=path_ddict)

In [182]:
for key in ddict['record_types']['HOUSING RECORD']:
    print("'{key}': __clude. {desc}. TODO.  ".format(
        key=key, desc=ddict['record_types']['HOUSING RECORD'][key]['description']))
    if key == 'FACRP':
        break

'RT': __clude. Record Type. TODO.  
'SERIALNO': __clude. Housing unit/GQ person serial number. TODO.  
'DIVISION': __clude. Division code. TODO.  
'PUMA00': __clude. Public use microdata area code (PUMA) based on Census 2000 definition for data collected prior to 2012. Use in combination with PUMA10.. TODO.  
'PUMA10': __clude. Public use microdata area code (PUMA) based on 2010 Census definition for data collected in 2012 or later. Use in combination with PUMA00.. TODO.  
'REGION': __clude. Region code. TODO.  
'ST': __clude. State Code. TODO.  
'ADJHSG': __clude. Adjustment factor for housing dollar amounts (6 implied decimal places). TODO.  
'ADJINC': __clude. Adjustment factor for income and earnings dollar amounts (6 implied decimal places). TODO.  
'WGTP': __clude. Housing Weight. TODO.  
'NP': __clude. Number of person records following this housing record. TODO.  
'TYPE': __clude. Type of unit. TODO.  
'ACR': __clude. Lot size. TODO.  
'AGS': __clude. Sales of Agriculture Produ

In [210]:
import pprint

In [213]:
for key in df.columns.values:
    if key in ddict['record_types']['HOUSING RECORD']:
        print(key)
        pprint.pprint(ddict['record_types']['HOUSING RECORD'][key])
    else:
        print(key)
        print("Not in data dictionary.")
    print()

insp
Not in data dictionary.

RT
OrderedDict([('length', '1'),
             ('description', 'Record Type'),
             ('var_codes',
              OrderedDict([('H', 'Housing Record or Group Quarters Unit')]))])

SERIALNO
OrderedDict([('length', '13'),
             ('description', 'Housing unit/GQ person serial number'),
             ('var_codes',
              OrderedDict([('2009000000001..2013999999999',
                            'Unique identifier')]))])

DIVISION
OrderedDict([('length', '1'),
             ('description', 'Division code'),
             ('var_codes',
              OrderedDict([('0', 'Puerto Rico'),
                           ('1', 'New England (Northeast region)'),
                           ('2', 'Middle Atlantic (Northeast region)'),
                           ('3', 'East North Central (Midwest region)'),
                           ('4', 'West North Central (Midwest region)'),
                           ('5', 'South Atlantic (South region)'),
                  

In [ ]:
cols_include = [
    'SERIALNO', 'PUMA00', 'PUMA10', 'ST', 'ADJHSG', 'ADJINC', 'WGTP', 'NP', 'TYPE', 'ACR', 'BATH', 'BDSP', 'BLD',
    'BUS', 'HFL']

Actions for included columns:
'SERIALNO': Use as index.  
'PUMA00', 'PUMA10', 'ST': Combine and lookup lat-lon coordinates from census.gov.  
'ADJHSG', 'ADJINC': Multiply against other columns to adjust for inflation. See https://www.census.gov/library/publications/2009/acs/pums.html App 5.
    'ADJHSG': CONP, ELEP, FULP, GASP, GRNTP, INSP, MHP, MRGP, SMOCP, RNTP, SMP, WATP
    'ADJINC': INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, WAGP
'WGTP': Work through user verification file with housing weights.  
'NP': Numerical. Map to float.  
'TYPE': Categorical. Map to median income.  
'ACR': Numerical. Map to float.  
'BATH': Combine with other columns.  
'BDSP': Numerical. Map to float.  
'BLD': Map to median income.  
'BUS': Map to float.  
'HFL': Include. Map house heating fuel to median income if correlated.  

'RWAT': Include. Map has hot/cold running water to float.  
'RWATPR': Include. Map has running water to float. Join into 'RWAT'.  
'SINK': Include. Map has sink to float.  
'SMP': Include. Map second+ mortgage payments to float. Adjust all with 'ADJHSG'.  
'STOV': Include. Has stove/range. Use 'KIT'.  
'TEL': Include. Map has telephone to float.  
'TEN': Include. Map tenure to median income if correlated.  
'TOIL': Include. Map has toilet to float.  
'VACS': Include. Map vacancy status to median income if correlated.  
'VALP': Include. Map property value to float.  
'VEH': Include. Map number of vehicles to float.  
'WATP': Include. Map water cost to float. Adjust \$3+ with 'ADJHSG'.  
'YBL': Include. Map when structure built to float.  
'FES': Include. Map family type to float.  
'FINCP': Include. Map family income to float. Adjust all with 'ADJINC'.  
'FPARC': Exclude. Family presence and age children. Use 'HUPAC'.  
'GRNTP': Include. Map gross rent to float. Adjust all with 'ADJHSG'.  
'GRPIP': Exclude. Gross rent percent income to float. We want to predict income.  
'HHL': Include. Map household language to float.  
'HHT': Include. Map household/family type to float.  
'HINCP': Include. Map household income to float. Adjust all with 'ADJINC'.  
'HUGCL': Include. Map household with grandparent and grandchildren to float.  
'HUPAC': Include. Map household presence and age of children to float.  
'HUPAOC': Exclude. Household presence and age of own children. Use 'HUPAC'.  
'HUPARC': Exclude. Household presence and age of related children. Use 'HUPAC'.  
'KIT': Include. Map has complete kitchen to float.  
'LNGI': Include. Map limited english to float.  
'MULTG': Exclude. Multigenerational household.
'MV': Include. Map when moved into house to float.  
'NOC': Include. Number of own children in household. Map to float.
'NPF': Include. Number of persons in family. Map to float.  
'NPP': Exclude. Grandparent headed household.  
'NR': Exclude. Presence of nonrelative in household.  
'NRC': Include. Number of related children in household. Map to float.  
'OCPIP': Exclude. Selected monthly owner costs as percentage of household income.  
'PARTNER': Include. Unmarried partner household.  
'PLM': Include. Complete plumbing facilities. Map to float. Compare to above.  
'PSF': Exclude. Presence of subfamilies in household.  
'R18': Include. Presence of persons under 18 years in household. Map to float.  
'R60': Exclude. Presence of persons 60+ years in household.  
'R65': Include. Presence of persons 60+ years in household. Use 'R65'.  
'RESMODE': Exclude. Response mode to ACS.  
'SMOCP': Include. Selected monthly owner costs. Map to float.  
'SMX': Include. Second+ mortgage status. Map to float.  
'SRNT': Exclude. Specified rent unit.  
'SVAL': Exclude. Specified value owner unit.  
'TAXP': Include. Property taxes. Map to float.  
'WIF': Include. Number of workers in family. Map to float.  
'WKEXREL': Include. Work experience of householder and spouse. Map to float.  
'WORKSTAT': Include. Work status of householder or spouse in family households. Map to float. Reduce detail and combine with another column.  
'FACRP': Exclude. Lot size allocation flag.  
...
'FYBLP': Exclude. When structure first built allocation flag.  
'WGTP1': Exclude. Housing Weight replicate 1.
...
'WGTP80': Exclude. Housing Weight replicate 1.

## Select features

**Notes:**
* Example consumer databases: http://www.consumerreports.org/cro/money/consumer-protection/big-brother-is-watching/overview/index.htm?rurl=http%3A%2F%2Fwww.consumerreports.org%2Fcro%2Fmoney%2Fconsumer-protection%2Fbig-brother-is-watching%2Foverview%2Findex.htm
* Random forests are scale invariant, so they can accommodate non-linear transformation.
* Cast all values to floats so that compatable with most algorithms and can use </> logic. Otherwise less informationally dense and may require deeper tree structure to find features.
* To "map to float" ('b' is N/A, mapped to 0; 1 is Yes; 2 is No; other values are special):  
```python
test = pd.DataFrame(data=[['b', 1.0], ['1', 1.0], ['2', 1.0], ['3', 1.1], ['4', 1.1]], columns=['COL', 'ADJ'])
tfmask = test['COL'].isin(['b'])
test.loc[tfmask, 'COL'] = 0.0
test['COL'] = test['COL'].astype(float)
print(test.dtypes)
test
```
* To "adjust for inflation":  
```python
test['ADJ'] *= 1e-6
tfmask = test['COL'] >= 3.0
test.loc[tfmask, 'COL'] *= test.loc[tfmask, 'ADJ']
test
```
* TODO: Remove vacant units ('NP') from data frame.
* TODO: Filter categorical variables from metadata (those without '..').

In [12]:
features = collections.OrderedDict()
for col in df.columns.values:
    features[col] = collections.OrderedDict()
    features[col]['include']

array(['insp', 'RT', 'SERIALNO', 'DIVISION', 'PUMA00', 'PUMA10', 'REGION',
       'ST', 'ADJHSG', 'ADJINC', 'WGTP', 'NP', 'TYPE', 'ACR', 'AGS',
       'BATH', 'BDSP', 'BLD', 'BUS', 'CONP', 'ELEP', 'FS', 'FULP', 'GASP',
       'HFL', 'MHP', 'MRGI', 'MRGP', 'MRGT', 'MRGX', 'REFR', 'RMSP',
       'RNTM', 'RNTP', 'RWAT', 'RWATPR', 'SINK', 'SMP', 'STOV', 'TEL',
       'TEN', 'TOIL', 'VACS', 'VALP', 'VEH', 'WATP', 'YBL', 'FES', 'FINCP',
       'FPARC', 'GRNTP', 'GRPIP', 'HHL', 'HHT', 'HINCP', 'HUGCL', 'HUPAC',
       'HUPAOC', 'HUPARC', 'KIT', 'LNGI', 'MULTG', 'MV', 'NOC', 'NPF',
       'NPP', 'NR', 'NRC', 'OCPIP', 'PARTNER', 'PLM', 'PSF', 'R18', 'R60',
       'R65', 'RESMODE', 'SMOCP', 'SMX', 'SRNT', 'SVAL', 'TAXP', 'WIF',
       'WKEXREL', 'WORKSTAT', 'FACRP', 'FAGSP', 'FBATHP', 'FBDSP', 'FBLDP',
       'FBUSP', 'FCONP', 'FELEP', 'FFSP', 'FFULP', 'FGASP', 'FHFLP',
       'FINSP', 'FKITP', 'FMHP', 'FMRGIP', 'FMRGP', 'FMRGTP', 'FMRGXP',
       'FMVP', 'FPLMP', 'FREFRP', 'FRMSP', 'FRNTMP', 'F

## Export ipynb to html

In [158]:
!date --rfc-3339='seconds'

2016-01-01 05:40:17+00:00


In [ ]:
path_ipynb = os.path.join(path_static, basename, basename+'.ipynb')
for template in ['basic', 'full']:
    path_html = os.path.splitext(path_ipynb)[0]+'-'+template+'.html'
    cmd = ['jupyter', 'nbconvert', '--to', 'html', '--template', template, path_ipynb, '--output', path_html]
    print(' '.join(cmd))
    subprocess.run(args=cmd, check=True)